# *IceFlow & icepyx*: Altimetry Time Series Tutorial
### NASA Earthdata Webinar - April 2021</b>

This tutorial demonstrates how to harmonize several NASA altimetry data products with varying temporal coverage, formats, and coordinate reference frames using the IceFlow and icepyx Python tools. 

#### Questions addressed:

#### Objectives:
1. Explore coverage and structure of the data...
2. Use IceFlow map widget to view area of interest...
3. Search and download spatiotemporally coincident data...
4. Use icepyx to subset ICESat-2 data...
5. Learn about advanced icepyx features...
6. Extract variables across all data products...
7. Visualize time series...

<b>Authors:</b><br />
<span style="font-size:larger;">Jessica Scheick</span>, *University of New Hampshire*, Durham, New Hampshire<br />
<span style="font-size:larger;">Nicholas Kotlinski & Amy Steiker</span>, *NASA National Snow and Ice Data Center DAAC*, Boulder, Colorado, USA

---

#### Running this tutorial locally

To run this notebook locally, you must first create a computing environment. The easiest way to do this is by downloading the [`environment.yml`](https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/binder/environment.yml) file from [NSIDC's GitHub Repository of Data Tutorials](https://github.com/nsidc/NSIDC-Data-Tutorials). Then, navigate to the directory where you've stored the file and run:
```
conda env create -n iceflow-env -f environment.yml
activate iceflow-env
```

Then open the notebook using your preferred viewer ([JupyterLab](https://jupyterlab.readthedocs.io/en/stable/getting_started/starting.html) is included in this environment). More detailed instructions on setting up environments with conda can be found [here](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-from-an-environment-yml-file).

This notebook can be downloaded from [GitHub](https://github.com/nsidc/NSIDC-Data-Tutorials/tree/main/notebooks).

<b>Other needs:</b><br />
* X Markdown on how to run the notebook locally (we might need to adjust the link for where the notebook is).
* IceFlow API examples
* icepyx examples
* Code to configure an ICESat-2 file to match IceFlow data dictionary

### 1. NASA's Earthdata Credentials

To access data using the *IceFlow* library and *icepyx* package, it is necessary to log into [Earthdata Login](https://urs.earthdata.nasa.gov/). To do this, enter your NASA Earthdata credentials in the next step after executing the following code cell.

**Note**: If you don't have NASA Earthdata credentials you will need to register first at the link above. An account is free and available to everyone!

In [ ]:
# This cell will prompt you for your Earthdata username and password. Press
from iceflow.ui import IceFlowUI
client = IceFlowUI()
client.display_credentials()

In [ ]:
# This cell will verify if your credentials are valid. 
# This may take a little while. If it fails, please try again.

authorized = client.authenticate()
if authorized is None:
    print('Earthdata Login not successful')
else:
    print('Earthdata Login successful!')

**Note:** If the output shows "You are logged into NASA Earthdata!", then you are ready to proceed!

---

### 2. Accessing and harmonizing data across missions
#### 2.1. Accessing Data with the *IceFlow* Access Widget
The *IceFlow* access widget is a user interface tool to visualize flightpaths from IceBridge, draw a region of interest, set spatio-temporal parameters and place data orders to the *IceFlow* API and *icepyx* package without the need to writing code.
The output of the operations performed in the widget can be seen in the log window (right-most icon at the bottom of your browser.) 
<img src='./img/log-icons.png'> or by selecting it on the View menu "Show log console"

**Note:** The access widget is currently stateless, so if you change any parameter you will have to redraw your bounding box or polygon.

In [ ]:
# Let's start with the user interface. Using 'horizontal' will add the widget inline.
client.display_map('horizontal', extra_layers=True)

#### 2.2. Accessing data with the *IceFlow* API

### ToDo: Need to explain what the itrf and epoch values are and which ones are best to utilize in conjunction with ICESat-2 analysis. 

### **Amy note:** Will we see the bbox above in the map? Or is there another way we can visualize the area?

In [ ]:
# Small example subset over Jakobshavn glacier:
my_params ={
    'datasets': ['ATM1B', 'GLAH06', 'ILVIS2', 'ATL06'],
    'start': '2008-01-01',
    'end': '2018-12-31',
    'bbox': '-49.787268,69.103805,-49.526401,69.218717',
    'itrf': 'ITRF2014',
    'epoch': '2014.1'
}

# returns a json dictionary, the request parameters and the order's response.
granules_metadata = client.query_cmr(params=my_params)

In [ ]:
orders = client.place_data_orders(params=my_params)
orders

#### Check Order Status
The following cell will show you the status of your data order. You can proceed in the notebook once all orders are "COMPLETE". If you proceed earlier only the completed data orders will be downloaded.

In [ ]:
for order in orders:
    status = client.order_status(order)
    print(order['dataset'], order['id'], status['status'])

### **Amy Note** :In my order email history, I get 5 emails, and they all say that 0 granules were processed. Yet I am able to download the data in the next step.

#### Download Data
Once all data orders are "COMPLETE", you can proceed downloading the data. The data are downloaded to the /data folder of this notebook directory.

In [ ]:
for order in orders:
    status = client.order_status(order)
    if status['status'] == 'COMPLETE':
        client.download_order(order)

#### 2.3. Downloading ICESat-2 data with *icepyx*
OR What's going on behind the scenes with *icepyx*

In [ ]:
# Begin overview of icepyx including visualzation options

---

### 3. Working with the data
Now that we have downloaded our data, we need to make sure that they are in a common format to do analysis across missions.

### **Amy Note**: We should explain the libraries we're importing briefly. I generally think it's better practice to put all imports at the very top of the notebook but I don't have any documentation on hand to back that up! In particular we shuold explain why wer're ignoring warnings (and this could probably be in the same code block as the previous block). 

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd
import geopandas as gpd
from iceflow.processing import IceFlowProcessing as ifp

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Pre-IceBridge ATM granule data
preib_gdf = ifp.to_geopandas('data/atm1b_data_2020-11-15T20-05.hdf5')
# ICESat granule data
glas_gdf = ifp.to_geopandas('data/twaties-test-GLAH06-2000-2010.h5')
# ICESat-2 granule data
is2_gdf = ifp.to_geopandas('data/processed_ATL06_20181214041627_11690105_003_01.h5')

Variable /gt1l/land_ice_segments/delta_time not found in data/processed_ATL06_20181214041627_11690105_003_01.h5.
Variable /gt1l/land_ice_segments/h_li not found in data/processed_ATL06_20181214041627_11690105_003_01.h5.
Variable /gt1l/land_ice_segments/latitude not found in data/processed_ATL06_20181214041627_11690105_003_01.h5.
Variable /gt1l/land_ice_segments/longitude not found in data/processed_ATL06_20181214041627_11690105_003_01.h5.
Variable /gt1r/land_ice_segments/delta_time not found in data/processed_ATL06_20181214041627_11690105_003_01.h5.
Variable /gt1r/land_ice_segments/h_li not found in data/processed_ATL06_20181214041627_11690105_003_01.h5.
Variable /gt1r/land_ice_segments/latitude not found in data/processed_ATL06_20181214041627_11690105_003_01.h5.
Variable /gt1r/land_ice_segments/longitude not found in data/processed_ATL06_20181214041627_11690105_003_01.h5.


In [7]:
# first, let's see what's in the harmonized dataframe and its shape.
display(preib_gdf.head(), preib_gdf.shape)

,latitude,longitude,elevation,geometry
time,,,,
2002-11-28 19:34:56.257,-75.276492,-105.394557,108.382004,POINT (-105.39456 -75.27649)
2002-11-28 19:34:56.257,-75.276455,-105.394511,108.391998,POINT (-105.39451 -75.27645)
2002-11-28 19:34:56.257,-75.276418,-105.394469,108.234001,POINT (-105.39447 -75.27642)
2002-11-28 19:34:56.257,-75.276381,-105.394433,108.156998,POINT (-105.39443 -75.27638)
2002-11-28 19:34:56.258,-75.276345,-105.394401,107.953003,POINT (-105.39440 -75.27635)


(436439, 4)

In [8]:
# we do the same for the ICESat/GLAS dataframe
display(glas_gdf.head(), glas_gdf.shape)

,latitude,longitude,elevation,geometry
time,,,,
2003-02-25 11:15:44.377777,-75.330202,-105.341199,236.777,POINT (-105.34120 -75.33020)
2003-02-25 11:15:44.402777,-75.328719,-105.342984,237.621,POINT (-105.34298 -75.32872)
2003-02-25 11:15:44.427777,-75.327238,-105.344776,236.192,POINT (-105.34478 -75.32724)
2003-02-25 11:15:44.452777,-75.325758,-105.346574,232.466,POINT (-105.34657 -75.32576)
2003-02-25 11:15:44.477777,-75.324280,-105.348378,226.501,POINT (-105.34838 -75.32428)


(42436, 4)

In [9]:
display(is2_gdf.head(), is2_gdf.shape)

,elevation,latitude,longitude,geometry
time,,,,
1970-01-01 00:00:00.029996359,NaN,69.218585,-49.575167,POINT (-49.57517 69.21859)
1970-01-01 00:00:00.029996359,NaN,69.218053,-49.575350,POINT (-49.57535 69.21805)
1970-01-01 00:00:00.029996359,369.441101,69.217695,-49.575468,POINT (-49.57547 69.21769)
1970-01-01 00:00:00.029996359,369.938751,69.217160,-49.575646,POINT (-49.57565 69.21716)
1970-01-01 00:00:00.029996359,365.230255,69.216982,-49.575707,POINT (-49.57571 69.21698)


(1118, 4)

---

Use code and example from https://nbviewer.jupyter.org/github/nicholas-kotlinski/2020_ICESat-2_Hackweek_Tutorials/blob/master/05.Geospatial_Analysis/shean_ICESat-2_hackweek_tutorial_GeospatialAnalysis_rendered.ipynb

## [Draft notebook for April 2021 Earthdata Webinar]

#### Learning Objectives (tbd)
* (goal from discussion) Introduce audience to the “Ice cubed” (ICESat/OIB/ICESat-2) missions and products
* (goal from discussion Promote icepyx and IceFlow as a way to address the need to access/harmonize multiple data products/resolutions/formats/structure


#### Webinar Outline
* Intro of harmonization challenge
    - Moving one level deeper into the need for time series analysis across disparate platforms
* Intro to the altimetry missions
    - Use the IceFlow intro notebook to guide this
    - Highlight the impressive time series
* Overview of IceFlow and icepyx
    - Audience and use cases of these tools
    - Icepyx genesis, open science and community development principles
        - working on that balance between community development and end use
    - IceFlow overview
    - How do these tools address challenges
        - IceFlow aims to make it easier to harmonize OIB/ICESat data with ICESat-2 using backend IceFlow API
* Use Case / Application
    - Leverage ICESat-2 hackweek topics
    - https://github.com/ICESAT-2HackWeek/geospatial-analysis/blob/master/shean_ICESat-2_hackweek_tutorial_GeospatialAnalysis.ipynb
* Short navigation of nsidc site, how to get to tools, notebook
    - Have Jennifer post links while we talk.
* Demos
    - **Use this notebook to pulls pieces of iceflow / icepyx using a specific science application**